In [ ]:
# !python -m pip install semantic-kernel==0.2.7.dev0
# !python -m pip install qdrant-client

In [ ]:

import json
# import requests
# import torch
import os
import json
# from transformers import AutoModel, AutoTokenizer
from qdrant_client.models import Distance, VectorParams
# from qdrant_client import models, QdrantClient
import semantic_kernel as sk
import asyncio
from semantic_kernel.connectors.ai.open_ai import AzureTextCompletion, AzureTextEmbedding, AzureChatCompletion


In [ ]:

kernel = sk.Kernel()

deployment, api_key, endpoint = sk.azure_openai_settings_from_dot_env()

kernel.add_text_completion_service("dv", AzureTextCompletion("text-davinci-003", endpoint, api_key))
kernel.add_text_embedding_generation_service("ada2", AzureTextEmbedding("text-embedding-ada-002", endpoint, api_key))

deployment, api_key, endpoint = sk.azure_openai_settings_from_dot_env()

# Create an instance of the AzureTextEmbedding class
embedding_generator = AzureTextEmbedding(deployment, endpoint, api_key)

# List of text samples you want to generate embeddings for
texts = ['This is a test of the emergency broadcast system. This is only a test. If this had been an actual emergency Id be running for my life']

# Define the async function to get embeddings
async def create_embedding(data):
    embeddings = await embedding_generator.generate_embeddings_async(data)
    return(embeddings)

# Run the asynchronous function
result = await create_embedding(texts)
print(result)


In [ ]:
# with open("your_json_file.json", "r") as json_file:
#     data = json.load(json_file)

In [ ]:
def load_json_from_file(file_path):
    with open(file_path, "r") as infile:
        data = json.load(infile)
    return data

In [ ]:
data = []
path_to_top = "../../../notebooks/"
path_to_READMEs = "data/READMEs/"

filename = "azure_sdk_readme_net_list.json"
README_path = os.path.join(path_to_top,path_to_READMEs,filename)
print(README_path) 
data = load_json_from_file(README_path)

filename = "azure_sdk_readme_rust_list.json"
README_path = os.path.join(path_to_top,path_to_READMEs,filename)
print(README_path) 
data = data + load_json_from_file(README_path)

filename = "azure_sdk_readme_java_list.json"
README_path = os.path.join(path_to_top,path_to_READMEs,filename)
print(README_path) 
data = data + load_json_from_file(README_path)

filename = "azure_sdk_readme_python_list.json"
README_path = os.path.join(path_to_top,path_to_READMEs,filename)
print(README_path)
data = data + load_json_from_file(README_path)

filename = "azure_sdk_readme_javascript_list.json"
README_path = os.path.join(path_to_top,path_to_READMEs,filename)
print(README_path)
data = data + load_json_from_file(README_path)



In [ ]:
len(data)

In [ ]:
 def makeObjectFromJSONList(data):
    #### This function takes a list of GitHub repos with READMEs and returns a dictionary of the READMEs and their descriptions
    #### The dictionary is in the format {README_URL: README_DESCRIPTION}
    #### The function is used to create a dictionary of READMEs to be added to a Semantic Memory

    github_files ={}
    for repo in data:
        # print("repo",repo)
        try:
            github_files[repo["repo_name"]] = repo
        except: 
            print(repo)
    return github_files

In [ ]:
readmeObj = makeObjectFromJSONList(data)
len(readmeObj.keys())


In [ ]:
from qdrant_client import models, QdrantClient
qdrant = QdrantClient(path=".")
qdrant.recreate_collection(
    collection_name="AzureSDKs",
    vectors_config=VectorParams(size=1536, distance=models.Distance.COSINE),
)

memory_collection_name = "AzureSDKs"
# print("Adding some GitHub file URLs and their descriptions to a volatile Semantic Memory.")
# print("This will take a few minutes.")

i = 0
for entry, value in readmeObj.items():
    split_name = value["repo_name"].split('/')
    name_after_slash = split_name[1] if len(split_name) > 1 else split_name[0]
    build_payload = {"SDK": name_after_slash, "link_to_repo": value["link_to_repo"], "language": value["language"], "README_text": value["README_text"]}
    # print("build_payload: ", build_payload)
    try:
        if len(value["README_text"]) < 100:
            pass
        else:
            embedding = await create_embedding(value["README_text"][:2000])
            # print("embedding: ", embedding)
            # print(embedding.tolist())
            vector_embedding = embedding.tolist()
            qdrant.upload_records(
                collection_name=memory_collection_name,
                records=[
                   models.Record(
                        id=i,
                        vector=vector_embedding[0],
                        payload=build_payload
                    ) 
                ]
            )
            print("  README {} embedded".format(i))
    except Exception as e:
        print(f"An exception occurred: {e}")
        print("failed with repo ", value["repo_name"])
    i += 1

Extract relevant information and construct vectors and payloads

Connect to the Qdrant API

Create a Qdrant collection

Upload the points (embeddings and payloads) to the created Qdrant collection